# `smlb` mini demonstration:<br>Compare different optimization techniques on the same response surface.

Scientific Machine Learning Benchmark:<br>
A benchmark of regression models in chem- and materials informatics.<br>
2019-2020, Citrine Informatics.

In [ ]:
import smlb

import numpy as np
import matplotlib.pyplot as plt

## Setup

Generate a stream of pseudo-random number generators.

In [ ]:
prng = smlb.Random(rng=0)
seeds = list(np.flip(prng.random.split(30)))

## The dataset: Friedman-Silverman (1989)

Load a 10-dimensional dataset

In [ ]:
from smlb.datasets.synthetic import FriedmanSilverman1989Data

In [ ]:
dataset = FriedmanSilverman1989Data(dimensions=10)

## Identity learner

First, test optimization algorithms against the Friedman-Silverman function itself. An `IdentityLearner` learns to perfectly reproduce the provided dataset (must be of type `VectorSpaceData`). The `ExpectedValue` scorer then returns the value of the function as the score. In this case we attempt to maximize the function.

In [ ]:
from smlb.learners import IdentityLearner

In [ ]:
model_identity = IdentityLearner(dataset)
score_ev = smlb.ExpectedValue(maximize=True)

## Optimizers

For this demonstrate, we compare three optimizers: a random sampler, differential evolution as implemented in Scipy, and dual annealing as implemented in Scipy. For the random optimizer we must specify the number of samples. Here we take 1000. To keep things comparable we also specify 1000 function evaluations for dual annealing, though the algorithm will finish out its current iteration when it passes that threshold. Differential evolution does not expose the number of function evaluations as a parameter, but we can set the number of iterations and find that 10 yields good results.

In [ ]:
from smlb.optimizers import RandomOptimizer, ScipyDifferentialEvolutionOptimizer, ScipyDualAnnealingOptimizer

In [ ]:
max_evals = 1e3
max_de_iters = 10
optimizers = [
    RandomOptimizer(num_samples=max_evals, rng=seeds.pop()),
    ScipyDifferentialEvolutionOptimizer(rng=seeds.pop(), maxiter=max_de_iters),
    ScipyDualAnnealingOptimizer(rng=seeds.pop(), maxfun=max_evals)
]
labels = [
    "Random Samples",
    "Differential Evolution",
    "Dual Annealing"
]

## Running the workflow

The `OptimizationTrajecotoryPlot` is the only evaluation currently implemented. It draws the median trajectory, shades the quantiles, and optionally draws the extremal results as well. Here we shade the 0.25 to the 0.75 quantile and choose to draw the best/worst trajectory at each point. We run 6 trials for each optimizer.

In [ ]:
from smlb.workflows import OptimizationTrajectoryComparison

In [ ]:
num_trials = 6
fig, ax = plt.subplots()
trajectory_plot = smlb.OptimizationTrajectoryPlot(
    target=ax,
    optimizer_names=labels,
    log_scale=True,
    quantile_width=0.5,
    show_extrama=True
)
workflow = OptimizationTrajectoryComparison(
    data=dataset,
    model=model_identity,
    scorer=score_ev,
    optimizers=optimizers,
    evaluations=[trajectory_plot,],
    num_trials=num_trials
)
workflow.run()
ax.set_title("Friedman-Silverman function (1989)")
ax.legend()
plt.show()

Dual annealing does the best, finding the optimum within a few dozen function evaluations. Differential evolution doesn't do much better than random sampling at first, but pulls ahead after a few hundred evaluations and eventually finds the optimum.

## Trained Learner

Next, we train a learner on some data drawn from the Friedman-Silverman function and optimize a score applied to that model.

In [ ]:
from smlb.learners import RandomForestRegressionSklearn

In [ ]:
model_rf = RandomForestRegressionSklearn(rng=seeds.pop(), uncertainties="naive")

In [ ]:
num_train = 50
sampler = smlb.RandomVectorSampler(size=num_train, rng=seeds.pop())
training_data = sampler.fit(dataset).apply(dataset)
model_rf.fit(training_data)

In this example we assume that the goal is to minimize the function. The lowest value in the training data is taken to be the target and we calculate the probability of exceeding that target. The goal of "minimize" in the score indicates that the score calculates the probability of being _below_ the target.

In [ ]:
min_value = min(training_data.labels())
score_pi = smlb.ProbabilityOfImprovement(target=min_value, goal="minimize")

## Running the workflow

Use the same optimizers as above and similar plotting settings. To demonstrate some different settings, here the plot is on a linear scale and does not include the extremal trajectories.

In [ ]:
num_trials = 6
fig, ax = plt.subplots()
trajectory_plot = smlb.OptimizationTrajectoryPlot(
    target=ax,
    optimizer_names=labels,
    log_scale=False,
    quantile_width=0.5,
    show_extrama=False
)
workflow = OptimizationTrajectoryComparison(
    data=dataset,
    model=model_rf,
    scorer=score_pi,
    optimizers=optimizers,
    evaluations=[trajectory_plot,],
    num_trials=num_trials
)
workflow.run()
ax.set_title("Friedman-Silverman function (1989)")
ax.legend()
plt.show()

Dual annealing is once again the best performer.